<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/WAN2_2_5B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 1. 環境準備
# ===============================
!apt -y install -qq aria2

# ===============================
# 2. ComfyUI 本体の取得
# ===============================
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI
%cd /content/ComfyUI
!git reset --hard c60dc4177c16d50da025bda4ac7fd513bb86e699
!pip install -r requirements.txt

# ===============================
# 3. カスタムノードの追加
# ===============================
%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git
!git clone https://github.com/rgthree/rgthree-comfy
%cd rgthree-comfy
!pip install -r requirements.txt

%cd /content/ComfyUI/custom_nodes
!git clone https://github.com/city96/ComfyUI-GGUF.git
%cd ComfyUI-GGUF
!pip install -r requirements.txt

# ===============================
# 4. Diffusion Models (Wan 2.2)
# ===============================
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors \
    -d /content/ComfyUI/models/diffusion_models -o wan2.2_ti2v_5B_fp16.safetensors

# ===============================
# 5. Text Encoders
# ===============================
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors \
    -d /content/ComfyUI/models/text_encoders -o umt5_xxl_fp8_e4m3fn_scaled.safetensors

# ===============================
# 6. VAE Models
# ===============================
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
    https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors \
    -d /content/ComfyUI/models/vae -o wan2.2_vae.safetensors


# ===============================
# 7. cloudflared (外部アクセス用)
# ===============================
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')


# ===============================
# 8. ComfyUI 起動＋URL出力
# ===============================
%cd /content/ComfyUI

# スレッドの開始
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 126666 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) 